<a href="https://colab.research.google.com/github/jaeohshin/PyTorch_Sklearn_rasbt/blob/main/pytorch_learning_feb_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 사용자 정의 층 만들기

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

In [13]:
class MyModule(nn.Module):
    def __init__(self):
        super().__init__()
        l1 = nn.Linear(2, 4)
        a1 = nn.ReLU()
        l2 = nn.Linear(4, 4)
        a2 = nn.ReLU()
        l3 = nn.Linear(4, 1)
        a3 = nn.Sigmoid()

        l = [l1, a1, l2, a2,l3, a3]
        self.module_list = nn.ModuleList(l)

    def forward(self, x):
        for f in self.module_list:
            x = f(x)
        return x

In [14]:
model = MyModule()

In [15]:
model

MyModule(
  (module_list): ModuleList(
    (0): Linear(in_features=2, out_features=4, bias=True)
    (1): ReLU()
    (2): Linear(in_features=4, out_features=4, bias=True)
    (3): ReLU()
    (4): Linear(in_features=4, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

In [16]:
class NoisyLinear(nn.Module):
    def __init__(self, input_size, output_size,
                 noise_stddev=0.1):
        super().__init__()
        w = torch.Tensor(input_size, output_size)
        self.w = nn.Parameter(w)
        nn.init.xavier_uniform_(self.w)
        b = torch.Tensor(output_size).fill_(0)
        self.b = nn.Parameter(b)
        self.noise_stddev = noise_stddev

    def forward(self, x, training=False):
        if training:
            noise = torch.normal(0.0, self.noise_stddev, x.shape)
            x_new = torch.add(x, noise)
        else:
            x_new = x
        return torch.add(torch.mm(x_new, self.w), self.b)

### MNIST 숫자 분류

In [26]:
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
image_path = './'


transforms = transforms.Compose([
    transforms.ToTensor()])

mnist_train_dataset = torchvision.datasets.MNIST(
    root = image_path, train=True,
    transform=transforms, download=True)

mnist_test_dataset = torchvision.datasets.MNIST(
    root = image_path, train=False,
    transform=transforms, download=True)

batch_size = 64
torch.manual_seed(1)

train_dl = DataLoader(mnist_train_dataset,
                      batch_size, shuffle=True)


## Make NN

In [44]:
hidden_units = [32, 16]
image_size = mnist_train_dataset[0][0].shape # print(image_size) = torch.Size([1, 28, 28])
input_size = image_size[0] * image_size[1] * image_size[2] # print(input_size)=784
all_layers= [nn.Flatten()]

for hidden_unit in hidden_units:
    layer = nn.Linear(input_size, hidden_unit)
    all_layers.append(layer)
    all_layers.append(nn.ReLU())
    input_size = hidden_unit

all_layers.append(nn.Linear(hidden_units[-1], 10))
model = nn.Sequential(*all_layers)
model

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=32, bias=True)
  (2): ReLU()
  (3): Linear(in_features=32, out_features=16, bias=True)
  (4): ReLU()
  (5): Linear(in_features=16, out_features=10, bias=True)
)

In [48]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
torch.manual_seed(1)
num_epochs = 20

for epoch in range(num_epochs):
    accuracy_hist_train = 0

    for x_batch, y_batch in train_dl:
        pred = model(x_batch)
        loss = loss_fn(pred, y_batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        is_correct = (
            torch.argmax(pred, dim=1) == y_batch
            ).float()
        accuracy_hist_train += is_correct.sum()
    
    accuracy_hist_train /= len(train_dl.dataset)

    print(f'에포크 {epoch} 정확도'
          f'{accuracy_hist_train: 4f}')

에포크 0 정확도 0.860817
에포크 1 정확도 0.932850
에포크 2 정확도 0.945217
에포크 3 정확도 0.952750
에포크 4 정확도 0.957917
에포크 5 정확도 0.961983
에포크 6 정확도 0.965183
에포크 7 정확도 0.967933
에포크 8 정확도 0.969917
에포크 9 정확도 0.972133
에포크 10 정확도 0.973933
에포크 11 정확도 0.975467
에포크 12 정확도 0.976583
에포크 13 정확도 0.978400
에포크 14 정확도 0.979000
에포크 15 정확도 0.979800
에포크 16 정확도 0.981983
에포크 17 정확도 0.982250
에포크 18 정확도 0.981867
에포크 19 정확도 0.983883


In [49]:
pred = model(mnist_test_dataset.data / 255.)
is_correct = (
    torch.argmax(pred, dim=1) == 
    mnist_test_dataset.targets).float()

print (f'Test accuracy: {is_correct.mean():.4f}')

Test accuracy: 0.9689
